In [21]:
import boto3
from time import gmtime, strftime
from sagemaker import get_execution_role
 
container = "arn:aws:sagemaker:eu-west-1:985815980388:algorithm/ifrs9-d7e2af4c3c2d4303eb43d3897ac79fd4"
maxRuntimeInSeconds = 1 * 60 * 60 # 1 hour

def get_sagemaker():
    region = boto3.Session().region_name
    return boto3.Session().client(service_name='sagemaker',region_name=region)

def create_ifrs9_job(sagemaker, 
                        container, 
                        s3_input, 
                        s3_output,
                        reporting_date,
                        loan_tape_path,
                        new_production_path,
                        matrix_path,
                        rating_grades_path,
                        macro_series,
                        macro_params,
                        segment_methodology_path,
                        ecl_max_number_of_periods = 5,
                        number_of_projection_periods = 3
                       ):
    role = get_execution_role()
    params = {
        "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
        "RoleArn": role,
        "OutputDataConfig": {"S3OutputPath": s3_output},
        "ResourceConfig": {
            "InstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            "VolumeSizeInGB": 5,
        },
        "HyperParameters": {
            "reporting_date": reporting_date, # TODO: validate format
            "loan_tape_path": loan_tape_path,
            "new_production_path": new_production_path,
            "matrix_path": matrix_path,
            "rating_grades_path": rating_grades_path,
            "macro_series": macro_series,
            "macro_params": macro_params,
            "segment_methodology_path": segment_methodology_path,
            "ecl_max_number_of_periods": f"{ecl_max_number_of_periods}",
            "number_of_projection_periods": f"{number_of_projection_periods}",
        },
        "StoppingCondition": {"MaxRuntimeInSeconds": maxRuntimeInSeconds},
        "InputDataConfig": [
            {
                "ChannelName": "source",
                "DataSource": {
                    "S3DataSource": {
                        "S3DataType": "S3Prefix",
                        "S3Uri": s3_input,
                        "S3DataDistributionType": "FullyReplicated",
                    }
                },
                "ContentType": "text/csv",
                "CompressionType": "None",
            },
        ],
    }

    training_job_name = "csow-1-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    params["TrainingJobName"] = training_job_name
    params["ResourceConfig"]["InstanceCount"] = 1

    job = sagemaker.create_training_job(**params)

    print(f"Created ifrs9 job: {training_job_name}")
    return training_job_name

def get_job_description(sagemaker, job_name):
    return sagemaker.describe_training_job(TrainingJobName=job_name)

def print_job_status(sagemaker, job_name):
    print(f'{job_name}: {get_job_description(sagemaker, job_name)["TrainingJobStatus"]}')

def wait_for_job_completion(sagemaker, job_name):
    print(f"Waiting for completion of {job_name}.")
    sagemaker.get_waiter("training_job_completed_or_stopped").wait(TrainingJobName=job_name)
    description = get_job_description(sagemaker, job_name)
    if description["TrainingJobStatus"] == "Failed":
        message = description["FailureReason"]
        print(f"Ifrs9 job {job_name} failed with the following error: {message}")
    else:
        print(f"Ifrs9 job {job_name} completed.")

In [22]:
s3_input = "s3://experiment-70357004-208c-4644-a08f-8859f253f372/dummy/Dart_x_v11/"
s3_output = "s3://experiment-70357004-208c-4644-a08f-8859f253f372/sage_out/"

sagemaker = get_sagemaker()

job_name = create_ifrs9_job(
    sagemaker, 
    container, 
    s3_input, 
    s3_output,
    "2015-12-31",
    "DART_loan_tape.csv",
    "DART_new_production.csv",
    "DART_MM.csv",
    "DART_risk_buckets.csv",
    "DART_macro_data.csv",
    "DART_macro_parameters.csv",
    "DART_segment_methodology.csv",
    5,
    3)
    
print_job_status(sagemaker, job_name)

wait_for_job_completion(sagemaker, job_name)

Created ifrs9 job: csow-1-2020-11-03-08-50-46
csow-1-2020-11-03-08-50-46: InProgress
Waiting for completion of csow-1-2020-11-03-08-50-46.
Ifrs9 job csow-1-2020-11-03-08-50-46 completed.
